In [12]:
!pip install protobuf

#üython 3.11.5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.2 MB/s eta 0:00:00a 0:00:01


In [2]:
# Imports
def doImport():
    global pd, sys, cdb
    try:
        import sys
        import pandas as pd
        import chromadb as cdb # force install pydantic version to 1.9.0 !!! otherwise this will throw PydanticImportError
        return "Successfully Imported All Modules"
    except Exception:
        # try to install missing modules
        sys.stderr.write("Missing modules detected, please install according to readme...\n")
    
doImport()

'Successfully Imported All Modules'

In [53]:
PERSISTENT_DB = False

model_name = "mrm8488--t5-base-finetuned-question-generation-ap"

metadata_fields = ["authors", "pubdate"] # field names that contain metadata
model_cache_dir = "cache/"
data_dir = "preprocessing/data/out"
firstNElements = 1000 # number of elements to process

In [4]:
# load every .txt file in data_dir in pandas dataframe with columns: "text", "ID". ID is the filename without extension | text is the text of the file
def load_data(data_dir):
    import os
    import pandas as pd
    data = []
    for filename in os.listdir(data_dir)[:firstNElements]:
        if filename.endswith(".txt"):
            with open(os.path.join(data_dir, filename), "r") as f:
                data.append({"text": f.read(), "ID": filename[:-4]})
    return pd.DataFrame(data)

In [5]:
from sentence_transformers import InputExample

data = load_data(data_dir)

def create_example(doc1: pd.Series):
    return InputExample(texts=[doc1])

train_examples_texts = data.apply(lambda x: create_example(x["text"]), axis=1).tolist()

In [6]:
from sentence_transformers import SentenceTransformer



model = SentenceTransformer(
    model_name,
    cache_folder=model_cache_dir
)

text_embeddings = model.encode(data["text"].tolist(), show_progress_bar=True)
print(f"Dimensions: {len(text_embeddings)}, {len(text_embeddings[0])}")

Batches: 100%|██████████| 32/32 [00:01<00:00, 16.47it/s]

Dimensions: 1000, 384


In [7]:
if PERSISTENT_DB:
    db_dir = "db/"
    db_name = "chroma.db"
    db_path = db_dir + db_name
    db = cdb.Client(cdb.config.Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=db_path))
else:
    chroma_client = cdb.Client()

Using embedded DuckDB without persistence: data will be transient


In [8]:
collection_name = "pubmed"

if(len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]):
    chroma_client.delete_collection(name=collection_name)
else:
    print(f"Creating Collection: {collection_name}")
    collection = chroma_client.create_collection(name=collection_name)
    

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Creating Collection: pubmed


In [9]:
collection.add(
    documents=data["text"].tolist(),
    # metadatas=[{metadata: datapoint} for metadata in metadata_fields for datapoint in data[metadata].tolist()],
    ids=data["ID"].tolist()
)

In [17]:
print(results)

{'ids': [['34131359', '33352270', '36068652', '34276140', '33649773', '35407664', '36981511', '32993795', '37455915', '32356867']], 'embeddings': None, 'documents': [['40. New Gener Comput. 2021;39(3-4):717-741. doi: 10.1007/s00354-021-00128-0. Epub\n 2021 Jun 10.\n\nLeveraging Artificial Intelligence (AI) Capabilities for COVID-19 Containment.\n\nSurianarayanan C(1), Chelliah PR(2).\n\nAuthor information:\n(1)Government Arts and Science College (Formerly Bharathidasan University \nConstituent Arts and Science College), Affiliated to Bharathidasan University, \nTiruchirappalli, Tamilnadu India.\n(2)Site Reliability Engineering Division, Reliance Jio Platforms Ltd, Bangalore, \nIndia.\n\nThe Coronavirus disease (COVID-19) is an infectious disease caused by the newly \ndiscovered Severe Acute Respiratory Syndrome Coronavirus two (SARS-CoV-2). Most \nof the people do not have the acquired immunity to fight this virus. There is no \nspecific treatment or medicine to cure the disease. The e

In [58]:
# naive q a system
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# model gpt would prefer: gpt2-xl

# model = "gpt2-xl"
model = "EleutherAI/gpt-j-6B"

tokenizer = AutoTokenizer.from_pretrained(model,
                                          cache_dir=model_cache_dir)
model = AutoModelForCausalLM.from_pretrained(model,
                                             cache_dir=model_cache_dir)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device_map="auto")

tokenizer_config.json: 100%|██████████| 619/619 [00:00<00:00, 7.38MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 10.9MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 11.5MB/s]
tokenizer.json: 100%|██████████| 1.37M/1.37M [00:00<00:00, 2.89MB/s]
added_tokens.json: 100%|██████████| 4.04k/4.04k [00:00<00:00, 29.9MB/s]
config.json: 100%|██████████| 930/930 [00:00<00:00, 6.88MB/s]
pytorch_model.bin:  34%|███▎      | 8.16G/24.2G [12:04<23:23, 11.4MB/s]

In [56]:
question = "What is the best treatment for covid?"
results = collection.query(query_texts=[question])
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
prompt_template = f"Relevant context: {context}\n\n\n\n\nThe user's question: {question}"

In [49]:
prompt_template

"Relevant context: #161. BMJ Open. 2021 Jan 22;11(1):e043012. doi: 10.1136/bmjopen-2020-043012.\n\nResearch Evaluation Alongside Clinical Treatment in COVID-19 (REACT COVID-19): \nan observational and biobanking study.\n\nBurke H(1)(2), Freeman A(3)(2), Dushianthan A(1)(2), Celinski M(1)(2), Batchelor \nJ(1)(2)(4)(5), Phan H(1)(5)(6), Borca F(4)(6), Kipps C(1)(2)(4)(6), Thomas \nGJ(7), Faust SN(1)(2)(4)(6)(8), Sheard N(2), Williams S(2), Fitzpatrick P(9), \nLanders D(9), Wilkinson T(1)(2).\n\nAuthor information:\n(1)Faculty of Medicine, University of Southampton, Southampton, UK.\n(2)University Hospital Southampton NHS Foundation Trust, Southampton, \nSouthampton, UK.\n(3)Faculty of Medicine, University of Southampton, Southampton, UK \na.freeman@soton.ac.uk.\n(4)Institute for Life Sciences, University of Southampton, Southampton, UK.\n(5)Clinical Informatics Research Unit, Faculty of Medicine, University of \nSouthampton, Southampton, UK.\n(6)NIHR Southampton Biomedical Research Centr

In [57]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (8950 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self